# ECS529U Algorithms and Data Structures                  
# Lab sheet 10

This lab gets you to work with dynamic programming algorithms and also practically 
compare their efficiency by testing them on randomly generated inputs. 

**Marks (max 5):**  Questions 1-5: 1 each

## Question 1

We define the *Thribonacci* sequence of numbers by the following function _thrib_:

- _thrib_(0) = 2
- _thrib_(1) = 1
- _thrib_(2) = 0
- _thrib_(n) = 3 _thrib_(n-1) + 2 _thrib_(n-2) + _thrib_(n-3) ,  if n > 2

Write a recursive Python function 

    def thrib(n)
    
that, on input `n`, returns _thrib_(`n`). 

Then, change your function into a dynamic programming one: 

    def thribDP(n)

using memoisation.

In [2]:
def thrib(n):
    if n == 0:
        return 2
    elif n == 1:
        return 1
    elif n == 2:
        return 0
    else:
        return 3 * thrib(n-1) + 2 * thrib(n-2) + thrib(n-3)


def thribDP(n):
    memo = {
        0: 2,
        1: 1,
        2: 0
    }
    return thribDPHelper(n, memo)


def thribDPHelper(n, memo):
    if n in memo:
        return memo[n]
    memo[n] = 3 * thribDPHelper(n-1, memo) + 2 * thribDPHelper(n-2, memo) + thribDPHelper(n-3, memo)
    return memo[n]


print(thrib(10),thribDP(10))
print(thrib(20),thribDP(20))
print(thrib(30),thribDP(30))
print(thribDP(50),thribDP(100))


29592 29592
11670435307 11670435307
4602562972405272 4602562972405272
715855072512917107789681077 6829493960642993121367570605611993654682546263670624652


## Question 2

Change your DP function from Question 1 into a dynamic programming bottom-up one:

    def thribDPBU(n)

using iteration

In [3]:
def thribDPBU(n):
    if n == 0:
        return 2
    elif n == 1:
        return 1
    elif n == 2:
        return 0
    
    bcs = [2, 1, 0]
    for i in range(3, n+1):
        bcs.append(3 * bcs[i-1] + 2 * bcs[i-2] + bcs[i-3])
    return bcs[n]


print(thribDPBU(10),thribDPBU(30))
print(thribDP(50),thribDP(100))

29592 4602562972405272
715855072512917107789681077 6829493960642993121367570605611993654682546263670624652


# Question 3

Write Python functions:

    def coinSplitTime(n)
    def coinSplitDPTime(n)
    def coinSplitDPBUTime(n)
    
that run `coinSplit(n)`, `coinSplitDP(n)` and `coinSplitDPBU(n)` respectively on input `n` 
and return the time taken for each of them to return. 

Test your timing functions on values 10, 100, 1000, 10000 for `n` and fill in the next table. 

Use these two choices for `coin`:

1. `coin1 = [200, 100, 50, 20, 5, 2, 1]`            
2. `coin2 = [200, 199, 198, ..., 3, 2, 1]`

To avoid waiting forever, if a run takes more than e.g. 15 seconds then kill it and fill in "timeout" in the table. **Note you need to fill in the table to get marks in this question!**

| value n/ coin array  |  10/ coin1 | 100/ coin1 | 1000/ coin1 | 10000/ coin1 | 10/ coin2 | 100/ coin2 | 1000/ coin2 | 10000/ coin2 |
|:------------|------|-----|------|-------|--------|--------|--------|--------|
| coinSplit time (sec)| 0.00001 | 0.00066 | 3.82458 | timeout | 0.00003 | timeout | timeout | timeout |               
| coinSplitDP time (sec)| 0.00002 | 0.00013 | 0.00064 | timeout | 0.00005 | 0.00144 | 0.05512 | timeout |                
| coinSplitDPBU time (sec)| 0.00002 | 0.00025 | 0.00128 | 0.02091 | 0.00016 | 0.00212 | 0.03651 | 0.48506 |                

In [1]:
import time
import signal

# add a counter so that we can check how many recursive calls are made
c1 = 0

def coinSplit(m):
    global c1
    c1 = 0
    return coinSplitRec(m,0)

def coinSplitRec(m, i): # split m using coins from coin[i:]
    global c1
    c1 += 1
    if m == 0:
        return 0
    if i == len(coin)-1:
        return m
    withoutIt = coinSplitRec(m,i+1)
    if coin[i] <= m:
        withIt = 1 + coinSplitRec(m-coin[i],i)
        if withIt < withoutIt:
            return withIt
    return withoutIt

# add a counter so that we can check how many recursive calls are made
c2 = 0

def coinSplitDP(m):
    global c2
    c2 = 0
    memo = {}
    return coinSplitMem(m,0,memo)

def coinSplitMem(m, i, memo):
    global c2
    c2 += 1    
    if (m,i) in memo:
        return memo[m,i]
    if m == 0:
        memo[m,i] = 0
    elif i == len(coin)-1:
        memo[m,i] = m
    else:
        withoutIt = coinSplitMem(m,i+1,memo)
        memo[m,i] = withoutIt
        if coin[i] <= m:
            withIt = 1 + coinSplitMem(m-coin[i],i,memo)
            if withIt < withoutIt:
                memo[m,i] = withIt
    return memo[m,i]


def coinSplitDPBU(mInit):
    memo = {}
    for i in range(len(coin)):
        memo[0,i] = 0
    for m in range(mInit+1):
        memo[m,len(coin)-1] = m
    for m in range(1,mInit+1):
        for i in range(len(coin)-2,-1,-1):
            withoutIt = memo[m,i+1]
            memo[m,i] = withoutIt
            if coin[i] <= m:
                withIt = 1 + memo[m-coin[i],i]
                if withIt < withoutIt:
                    memo[m,i] = withIt
    return memo[mInit,0]


def timeout_handler(signum, frame):
    raise Exception("Timeout")


def coinSplitTime(n):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(15)
    start_time = time.time()
    try:
        coinSplit(n)
        return time.time() - start_time
    except Exception as e:
        return 15
    finally:
        signal.alarm(0)


def coinSplitDPTime(n):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(15)
    start_time = time.time()
    try:
        coinSplitDP(n)
        return time.time() - start_time
    except Exception as e:
        return 15
    finally:
        signal.alarm(0)

def coinSplitDPBUTime(n):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(15)
    start_time = time.time()
    try:
        coinSplitDPBU(n)
        return time.time() - start_time
    except Exception as e:
        return 15
    finally:
        signal.alarm(0)





In [2]:
# Test code for Question 3

# Define the coin arrays
coin1 = [200, 100, 50, 20, 5, 2, 1]
coin2 = list(range(200, 0, -1))  # [200, 199, 198, ..., 3, 2, 1]

# Test values
test_values = [10, 100, 1000, 10000]

# Results storage
results = {
    'coinSplit': {'coin1': [], 'coin2': []},
    'coinSplitDP': {'coin1': [], 'coin2': []},
    'coinSplitDPBU': {'coin1': [], 'coin2': []}
}

coin = coin1
for n in test_values:
    results['coinSplit']['coin1'].append((coinSplitTime(n), n))
    results['coinSplitDP']['coin1'].append((coinSplitDPTime(n), n))
    results['coinSplitDPBU']['coin1'].append((coinSplitDPBUTime(n), n))

coin = coin2
for n in test_values:
    results['coinSplit']['coin2'].append((coinSplitTime(n), n))
    results['coinSplitDP']['coin2'].append((coinSplitDPTime(n), n))
    results['coinSplitDPBU']['coin2'].append((coinSplitDPBUTime(n), n))

# print the results:
for algo in results:
    print(f"\nResults for {algo}:")
    for coin_key in results[algo]:
        print(f"  Using {'coin1' if coin_key == 'coin1' else 'coin2'}:")
        for time_taken, n in results[algo][coin_key]:
            print(f"    n = {n}: {time_taken:.5f} seconds")





Results for coinSplit:
  Using coin1:
    n = 10: 0.00001 seconds
    n = 100: 0.00066 seconds
    n = 1000: 3.82458 seconds
    n = 10000: 15.00000 seconds
  Using coin2:
    n = 10: 0.00003 seconds
    n = 100: 15.00000 seconds
    n = 1000: 15.00000 seconds
    n = 10000: 15.00000 seconds

Results for coinSplitDP:
  Using coin1:
    n = 10: 0.00002 seconds
    n = 100: 0.00013 seconds
    n = 1000: 0.00064 seconds
    n = 10000: 15.00000 seconds
  Using coin2:
    n = 10: 0.00005 seconds
    n = 100: 0.00144 seconds
    n = 1000: 0.05512 seconds
    n = 10000: 15.00000 seconds

Results for coinSplitDPBU:
  Using coin1:
    n = 10: 0.00002 seconds
    n = 100: 0.00025 seconds
    n = 1000: 0.00128 seconds
    n = 10000: 0.02091 seconds
  Using coin2:
    n = 10: 0.00016 seconds
    n = 100: 0.00212 seconds
    n = 1000: 0.03651 seconds
    n = 10000: 0.48506 seconds


# Question 4

We study the problem of finding the longest common subsequence between two DNA sequences. For us, DNA sequences will be strings with characters taken from the four DNA bases (`A`, `G`, `C`, `T`). Write a dynamic programming function

    def maxDNAsubseq(s, t)
    
that takes input strings `s` and `t` and returns a common substring thereof of maximum length. For example,
- if `s=ACCGGTCGAGTGCGCGGAAGCCGGCCGAA` and `t=GTCGTTCGGAATGCCGTTGCTCTGTAAA` then `maxDNAsubseq(s,t)` should return `GTCGTCGGAAGCCGGCCGAA`;
- if `s=ABCBDAB` and `t=BDCABA` then `maxDNAsubseq(s,t)` should return `BCBA` or `BDAB`.

*Hint:* Work similarly to the longest palindrome problem from Lecture 10. You can use following property. If `s` and `t` are not empty then:
- if `s[0]==t[0]` then the longest common substring of `s` and `t` must contain their initial character;
- otherwise, the longest common substring of `s` and `t` can be obtained by comparing the longest common substring of `s[1:]` and `t`  with that of `s` and `t[1:]`.

In [7]:
def maxDNAsubseq(s, t):
    memo = {}
    return lcsMem(s, t, 0, 0, memo)

def lcsMem(s, t, i, j, memo):
    if i == len(s) or j == len(t):
        return ""
    
    if (i, j) in memo:
        return memo[i, j]
    
    if s[i] == t[j]:
        result = s[i] + lcsMem(s, t, i+1, j+1, memo)
    else:
        option1 = lcsMem(s, t, i+1, j, memo)
        option2 = lcsMem(s, t, i, j+1, memo)
        if len(option1) > len(option2):
            result = option1
        else:
            result = option2
    
    memo[i, j] = result
    return result

print(maxDNAsubseq("ACCGGTCGAGTGCGCGGAAGCCGGCCGAA","GTCGTTCGGAATGCCGTTGCTCTGTAAA"))
print(maxDNAsubseq("ABCBDAB","BDCABA"))

GTCGTCGGAAGCCGGCCGAA
BCBA


# Question 5

Using dynamic programming, write a Python function: 

    def closestSubset(A,s)

that takes an array of non-negative integers `A` and a non-negative integer `s` and returns an array consisting 
of elements of `A` (i.e. a subset of `A`) which add up to `s`. If there is no subset that adds up to `s`, the function 
should instead return a subset which adds up to the value closest to `s`. 

For example: 

- if `A` is `[12, 79, 99, 91, 81, 47]` and `s` is `150`, it will return `[12, 91, 47]` as 12+91+47 is 150
- if `A` is `[15, 79, 99, 6, 69, 82, 32]` and `s` is `150` it will return `[69, 82]` as 69+82 is 151, and 
there is no subset of `A` whose sum is 150.

In more detail, your function should use an auxiliary function:

    def closestSubsetMem(A,s,lo,memo)
    
that returns an array consisting of elements of `A[lo:]` which add up to the closest value to `s`. To implement this function, you can use recursion as follows:

- If `s` is less or equal to 0 or `lo` is beyond the bounds of `A`, then the solution is simply `[]`
- Otherwise, we first consider the (recursive) case where element `A[lo]` is included in the selected elements (case withIt): 

    - if this case returns an array of elements that add up to `s` then the solution is that array
    - otherwise, we also consider the (recursive) case where element `A[lo]` is not included in the selected elements (case withoutIt); between the returned arrays of cases withIt and withoutIt, we select the one whose elements sum up closer to `s`.

Test the method with arrays generated by `randomIntArray(s,n)` from Lab 3. Try with:

    A = randomIntArray(20,1000)
    subset = closestSubset(A,5000)

In [16]:
import random

def randomIntArray(s, n):
    return [random.randint(0, s) for _ in range(n)]

def closestSubset(A, s):
    memo = {}
    return closestSubsetMem(A, s, 0, memo)

def closestSubsetMem(A, s, lo, memo):
    if s <= 0 or lo >= len(A):
        return []
    
    if (s, lo) in memo:
        return memo[s, lo]
    
    with_it = closestSubsetMem(A, s-A[lo], lo+1, memo)
    with_it = with_it + [A[lo]]
    without_it = closestSubsetMem(A, s, lo+1, memo)

    sum_with_it = sum(with_it) if with_it else 0
    sum_without_it = sum(without_it) if without_it else 0
    
    if sum_with_it == s:
        result = with_it
    else:
        dist_with_it = abs(s - sum_with_it)
        dist_without_it = abs(s - sum_without_it)
        
        if dist_with_it < dist_without_it:
            result = with_it
        else:
            result = without_it
    
    memo[(s, lo)] = result
    return result

def addAll(A):
    return sum(A)

A=[12, 79, 99, 91, 81, 47]
S=closestSubset(A,150)
print("Closest subset of",A,"for sum",150,"is",S,"with sum",addAll(S))
A=[15, 79, 99, 6, 69, 82, 32] 
S=closestSubset(A,150)
print("Closest subset of",A,"for sum",150,"is",S,"with sum",addAll(S))

for i in range(10):
    A = randomIntArray(20,1000)
    subset = closestSubset(A,5000)
    print("Closest subset of",len(A),"elements for sum",5000,"is",subset,"with sum",addAll(subset))


Closest subset of [12, 79, 99, 91, 81, 47] for sum 150 is [47, 91, 12] with sum 150
Closest subset of [15, 79, 99, 6, 69, 82, 32] for sum 150 is [82, 69] with sum 151
Closest subset of 1000 elements for sum 5000 is [4, 9, 1, 11, 16, 9, 11, 3, 10, 19, 1, 1, 4, 7, 14, 19, 4, 6, 20, 16, 18, 13, 13, 7, 16, 0, 12, 19, 6, 20, 10, 0, 0, 12, 14, 20, 0, 8, 18, 9, 13, 12, 13, 8, 0, 17, 7, 5, 9, 0, 14, 10, 7, 3, 7, 5, 1, 4, 2, 16, 15, 17, 5, 12, 17, 6, 10, 13, 2, 6, 2, 7, 1, 3, 11, 18, 0, 14, 15, 12, 19, 12, 11, 9, 14, 18, 18, 16, 15, 5, 17, 7, 5, 3, 17, 9, 14, 1, 14, 2, 5, 1, 9, 2, 9, 20, 9, 17, 1, 18, 16, 2, 8, 16, 16, 16, 8, 17, 10, 2, 19, 0, 3, 16, 1, 6, 19, 8, 11, 9, 14, 3, 7, 3, 0, 13, 16, 13, 19, 8, 16, 0, 12, 18, 1, 3, 14, 18, 16, 20, 20, 1, 9, 12, 12, 3, 6, 14, 16, 19, 17, 19, 11, 10, 8, 7, 6, 18, 10, 13, 20, 1, 1, 14, 6, 9, 8, 15, 8, 6, 18, 16, 13, 7, 6, 8, 14, 0, 15, 13, 1, 20, 14, 0, 1, 5, 7, 12, 0, 19, 12, 12, 19, 20, 15, 10, 13, 14, 13, 11, 3, 15, 1, 19, 20, 13, 20, 2, 8, 10, 0, 3, 